## Ensembling models 

#### Performing Sampling with replacement on Training Data with sample size of 10000 

 Considering a Total Dataset of 1000000 points

In [68]:
D = final_xtr[0:1000000]

Splitting into 80% Train set & 20% Test Set

In [69]:
D_train_size = int(len(D) * 0.80)
D_train, D_test = D[0:D_train_size], D[D_train_size:len(D)]

Splitting D_train into 50% as D! & 50% as D2

In [70]:
D1_size = int(len(D_train) * 0.50)
D1, D2 = D_train[0:D1_size], D_train[D1_size:len(D_train)]

In [71]:
D1y = D1['meter_reading_log1p']

In [72]:
D2y = D2['meter_reading_log1p']

In [73]:
Dtest_y = D_test['meter_reading_log1p']

In [74]:
D1 = D1.drop(columns=['meter_reading', 'meter_reading_log1p'])

In [75]:
D2 = D2.drop(columns=['meter_reading', 'meter_reading_log1p'])

In [76]:
D_test = D_test.drop(columns=['meter_reading', 'meter_reading_log1p'])

In [77]:
print(D1.shape)
print(D2.shape)
print(D_test.shape)

(400000, 40)
(400000, 40)
(200000, 40)


#### Creating 1000 samples from D1 with each sample having a size of 10000 

In [99]:
from sklearn.utils import resample
array_x=[]
array_y=[]
for i in range(1000):
    x, y = resample(D1, D1y, random_state=i, n_samples = 10000)
    array_x.append(x)
    array_y.append(y)

#### Training 1000 Base learners with LGBM 

In [100]:
params = {
    "objective": "regression",
    "boosting": "gbdt",
    "num_leaves": 800,
    "learning_rate": 0.03,
    "feature_fraction": 0.8,
    "reg_lambda": 5, 
    "metric": "rmse",
}

#### Fitting 1000 samples & making predicitons on D2 as well as Test set 

In [101]:
D2_pred_df=[]
test_pred_df=[]

'''This function fits data & makes predictions then saves it in a DataFrame '''

def fit_pred(x, y, D2=D2, test_data=D_test, model=lgbm.LGBMRegressor(**params)):
    
        model = model.fit(x, y)
        D2_pred = model.predict(D2)
        test_pred = model.predict(D_test)
        D2_pred_df_1 = pd.DataFrame((D2_pred))
        test_pred_df_1 = pd.DataFrame((test_pred))
        D2_pred_df.append(D2_pred_df_1)
        test_pred_df.append(test_pred_df_1)

In [102]:
for i in range(1000):
    fit_pred(array_x[i], array_y[i], D2=D2, test_data=D_test, model=lgbm.LGBMRegressor(**params))

#### Concatenating all the predictions to form a DataFrame of Predicted Values

In [103]:
D2_pred = pd.DataFrame(D2_pred_df[0])
for i in range(999):
    D2_pred = pd.concat([D2_pred, D2_pred_df[i+1]], axis=1)

In [104]:
test_pred = pd.DataFrame(test_pred_df[0])
for i in range(999):
    test_pred = pd.concat([test_pred, test_pred_df[i+1]], axis=1)

In [105]:
D2_pred

,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,5.406890,4.676831,5.194169,4.178576,5.545815,5.493564,4.559661,5.562041,4.831146,5.211310,...,4.893570,5.250194,4.974121,5.332544,5.533834,4.843594,4.967505,5.290789,4.991265,4.695826
1,4.789692,4.799530,4.467926,5.300593,4.569659,5.432114,4.781309,4.931889,4.780574,4.591454,...,4.548010,4.918019,4.729093,4.871251,4.681666,5.066974,4.810571,4.859555,5.157564,4.671045
2,4.366646,4.786084,4.297739,4.771034,4.803522,4.605854,4.791783,4.592001,4.594080,4.493253,...,4.239320,4.468863,4.803478,4.672406,4.530058,4.887118,4.832490,4.776520,4.679661,4.584151
3,3.912403,2.837564,3.468420,3.410372,3.306435,4.115559,4.554656,3.099567,4.619218,4.277430,...,3.920322,3.755373,3.822495,3.966327,3.630068,3.427063,4.176191,3.102986,4.470023,3.550705
4,3.759165,3.158892,3.839723,4.016388,3.893071,3.527043,3.826708,4.172214,4.024904,4.402679,...,3.235085,4.334998,4.043826,3.725020,4.055716,3.909643,3.953403,3.974847,4.037254,4.191863
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,4.336449,4.273035,4.203988,3.684921,4.288318,2.678922,4.991395,4.652082,4.425072,4.734588,...,4.009313,5.144730,5.336997,4.931485,4.371347,3.756637,4.671143,4.127221,3.896071,3.163067
399996,3.466616,3.355224,3.605666,3.328009,3.488745,2.698223,3.405883,3.007074,3.661641,3.246319,...,3.487070,3.755997,3.687489,3.162192,3.359804,3.454168,3.530466,3.157016,3.630160,3.783548
399997,4.418525,4.028820,5.265522,4.571856,4.867911,4.212261,4.766486,4.687352,5.174721,5.387937,...,4.168133,5.630346,5.211793,4.965116,4.851971,4.385410,5.000381,4.493340,3.978167,3.714197
399998,4.500630,4.454337,4.424156,4.661215,4.532431,4.414233,4.635461,4.205366,4.802977,4.623285,...,4.318945,4.264591,4.212317,4.104954,4.518453,4.197425,4.352279,4.520133,4.493985,4.274006


In [106]:
test_pred

,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,4.727845,5.417161,5.854926,4.808374,5.633997,4.573354,5.949672,4.612934,4.716130,4.897125,...,4.981907,5.132680,5.896355,5.424908,5.254892,5.355361,5.691735,4.721754,4.454271,5.443764
1,7.474527,6.343790,7.470690,6.784343,7.410874,6.546529,6.073743,6.022096,6.705705,5.941748,...,6.227260,6.778938,7.154992,7.613619,6.959066,6.890251,7.610331,6.293382,6.712299,5.921689
2,4.270240,4.499316,4.768082,4.624256,4.652767,4.620062,4.365222,4.495624,4.959059,4.905536,...,4.376218,4.615318,4.346806,4.373204,4.650491,4.437957,4.577650,4.624518,4.294511,4.786368
3,2.682829,2.866852,3.356389,3.968579,3.397535,3.370532,2.627758,3.213573,3.818444,2.425537,...,2.877457,3.144647,2.669209,2.583436,3.938599,3.800186,3.187400,3.147807,2.922814,4.221638
4,4.211944,4.723949,5.590540,5.137603,6.332336,4.992555,4.851891,4.199922,4.829161,6.316487,...,4.058975,5.677038,5.143606,4.948030,4.536964,5.365379,4.903311,5.516043,3.854869,5.419966
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,5.983210,6.720848,6.438583,6.371096,6.635252,6.991361,7.067873,6.190016,7.078602,6.806992,...,6.822481,7.214806,6.471556,6.631453,5.651428,6.494923,6.418945,6.294427,7.165361,6.545907
199996,2.247749,3.880316,2.612840,4.073130,4.865105,4.022435,4.453422,4.437582,2.872044,5.410846,...,3.827830,3.145103,3.577875,3.865959,5.109585,3.496965,2.017558,4.282209,5.052475,4.213477
199997,7.450871,7.577063,7.124444,7.055494,7.471737,7.629081,6.521905,7.323249,7.462433,7.789647,...,7.375754,7.578001,7.336177,8.013768,7.768514,7.387824,6.922495,7.596874,7.537573,6.981371
199998,4.118501,5.105231,4.762551,5.368031,5.911019,6.327426,6.613160,6.128956,5.361384,6.310145,...,4.290434,5.619237,5.005396,4.663025,4.150372,5.282868,6.252950,5.181225,4.047866,4.311078


#### Removing Headers from the Predictions Dataset

In [107]:
D2_pred.to_csv('D2_pred.csv', header=False, index=False)
D2_pred = pd.read_csv('D2_pred.csv', header=None)
D2_pred = D2_pred.values

In [108]:
test_pred.to_csv('test_pred.csv', header=False, index=False)
test_pred = pd.read_csv('test_pred.csv', header=None)
test_pred = test_pred.values

#### Fitting on the Predictions DataFrame

In [109]:
lgbm_model = lgbm.LGBMRegressor(**params)
lgbm_model.fit(D2_pred, D2y)

LGBMRegressor(boosting='gbdt', boosting_type='gbdt', class_weight=None,
              colsample_bytree=1.0, feature_fraction=0.8,
              importance_type='split', learning_rate=0.03, max_depth=-1,
              metric='rmse', min_child_samples=20, min_child_weight=0.001,
              min_split_gain=0.0, n_estimators=100, n_jobs=-1, num_leaves=800,
              objective='regression', random_state=None, reg_alpha=0.0,
              reg_lambda=5, silent=True, subsample=1.0,
              subsample_for_bin=200000, subsample_freq=0)

#### Making Predictions on the Test Set 

In [110]:
preds = np.expm1(lgbm_model.predict(test_pred))

#### Converting D2y & Dtest_y to numpy array for getting Final scores

In [90]:
D2y = D2y.to_frame()

In [91]:
D2y = D2y.values

In [92]:
type(D2y)

numpy.ndarray

In [93]:
type(Dtest_y)

pandas.core.series.Series

In [94]:
Dtest_y = Dtest_y.to_frame()

In [95]:
Dtest_y = Dtest_y.values

In [96]:
type(Dtest_y)

numpy.ndarray

In [111]:
print('Train RMSLE = ',rmsle_score(np.expm1(D2y) , np.expm1(lgbm_model.predict(D2_pred))))

Train RMSLE =  0.48506846502713324


In [112]:
print('Test RMSLE = ',rmsle_score(np.expm1(Dtest_y) , preds),'\n')

Test RMSLE =  0.8357900830166635 

